In [3]:
# Analysing how teams when they are the away team on a B2B game set, compared to their season W/L

In [4]:
from basketball_reference_scraper.seasons import get_schedule, get_standings
from datetime import timedelta
import pandas as pd
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
import numpy as np
import matplotlib.pyplot as plt


In [5]:
start_year = 2000
end_year = 2013
df_columns = ['year', 'team', 'b2b_away_pct', 'b2b_away_wins', 'b2b_away_games', 'season_WL_pct', 'diff']
final_dict = {}
flat_final = pd.DataFrame(columns = df_columns)


In [10]:
for year in range(start_year, end_year, 1):
    all_results = get_schedule(year, playoffs=False)
    teams = all_results.HOME.unique().tolist()
    
    date = str(year) + '-06-06'
    standings = get_standings(date=date)
    combined_standings = pd.concat([standings['EASTERN_CONF'], standings['WESTERN_CONF']])
    combined_standings.TEAM = combined_standings.TEAM.str.rstrip('*')

    final_dict[year] = {}    
    
    for team in teams:
    #team = 'Miami Heat'
    
        heat_home = all_results[all_results.HOME.eq(team)]
        heat_away = all_results[all_results.VISITOR.eq(team)]
        
        heat = all_results[(all_results['HOME']==team) | (all_results['VISITOR']==team)]
        
        num_games = heat.shape[0]
        
        heat["GAME_BEFORE"] = [False] * num_games
        
        for i in range (0, num_games - 1):
            if (heat.DATE.iloc[i] + timedelta(days = 1) == heat.DATE.iloc[i + 1]):
                heat.GAME_BEFORE.iloc[i] = True
                heat.GAME_BEFORE.iloc[i + 1] = True
        
        heat_b2b = heat[(heat['GAME_BEFORE'] == True)]
        
        heat_b2b_away = heat_b2b[(heat['VISITOR'] == team)]
        heat_b2b_home = heat_b2b[(heat['HOME'] == team)]
        
        num_b2b = heat_b2b.shape[0]
        
        #new df, same structure
        b2b_away = pd.DataFrame(data=None, columns=heat_b2b.columns)
        
        #go thru home, look to see if there is a date + 1 in away, if there is get that data in b2b away
        num_b2b_home = heat_b2b_home.shape[0]
        
        for j in range(0, num_b2b_home):
        
            date = heat_b2b_home.DATE.iloc[j]
            date_pp = pd.Timestamp(date + timedelta(days = 1))
            
            if (date_pp in heat_b2b_away.DATE.tolist()):
                temp = pd.DataFrame(heat_b2b_away[heat_b2b_away['DATE'] == date_pp])
                b2b_away = pd.concat([b2b_away, temp])
                
        num_b2b_away = b2b_away.shape[0]
        num_b2b_away_wins = 0
        
        for i in range(0, num_b2b_away):
            vis_pts = b2b_away.VISITOR_PTS.iloc[i]
            home_pts = b2b_away.HOME_PTS.iloc[i]
            
            if (vis_pts > home_pts):
                num_b2b_away_wins += 1
                
        if (num_b2b_away != 0):
            b2b_away_pct = round(num_b2b_away_wins / num_b2b_away, 2)
        else:
            b2b_away_pct = 0 #0 or null?
            
        temp = combined_standings[combined_standings.TEAM.eq(team)]
        WL_perc = round(float(temp['W/L%']), 2)
        difference = round(b2b_away_pct - WL_perc, 2)
        final_dict[year][team] = {'b2b_away_pct' : b2b_away_pct,
                  'b2b_away_wins' : num_b2b_away_wins,
                  'b2b_away_games' : num_b2b_away,
                  'season_WL_pct' : WL_perc,
                  'diff' : difference}

        temp_dict = final_dict[year][team]   
        
        temp_dict['year'] = year
        temp_dict['team'] = team
        
        temp_df = pd.DataFrame(temp_dict, index = [0])
        
        cols = temp_df.columns.tolist()
        
        cols = cols[-1:] + cols[:-1]
        cols = cols[-1:] + cols[:-1]
        
        temp_df = temp_df[cols]
        flat_final = pd.concat([temp_df, flat_final])

/Users/Nillan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/Nillan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/Nillan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [11]:
flat_final.head(5)

,b2b_away_games,b2b_away_gp,b2b_away_pct,b2b_away_wins,diff,season_WL_pct,team,year
0,9.0,NaN,0.44,4,-0.09,0.53,Philadelphia 76ers,2012
0,3.0,NaN,1.00,3,0.24,0.76,Chicago Bulls,2012
0,3.0,NaN,0.67,2,0.06,0.61,Los Angeles Clippers,2012
0,7.0,NaN,0.43,3,-0.12,0.55,Utah Jazz,2012
0,6.0,NaN,0.17,1,-0.42,0.59,Boston Celtics,2012


In [12]:
from sklearn import preprocessing

normalized = flat_final.copy()

normalized.iloc[:, 2:-1].values

x = normalized.iloc[:, 2:-1].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
normalized = pd.DataFrame(x_scaled, columns = df_columns[2:-1])
corr = normalized.corr()

ValueError: could not convert string to float: 'Philadelphia 76ers'

In [22]:
normalized.iloc[:, 2:-1].values #returns a numpy array

normalized.head(5)

,b2b_away_games,b2b_away_gp,b2b_away_pct,b2b_away_wins,diff,season_WL_pct,team,year
0,9.0,NaN,0.44,4,-0.09,0.53,Philadelphia 76ers,2012
0,3.0,NaN,1.00,3,0.24,0.76,Chicago Bulls,2012
0,3.0,NaN,0.67,2,0.06,0.61,Los Angeles Clippers,2012
0,7.0,NaN,0.43,3,-0.12,0.55,Utah Jazz,2012
0,6.0,NaN,0.17,1,-0.42,0.59,Boston Celtics,2012


In [18]:
normalized.head(5)

,b2b_away_games,b2b_away_gp,b2b_away_pct,b2b_away_wins,diff,season_WL_pct,team,year
0,9.0,NaN,0.44,4,-0.09,0.53,Philadelphia 76ers,2012
0,3.0,NaN,1.00,3,0.24,0.76,Chicago Bulls,2012
0,3.0,NaN,0.67,2,0.06,0.61,Los Angeles Clippers,2012
0,7.0,NaN,0.43,3,-0.12,0.55,Utah Jazz,2012
0,6.0,NaN,0.17,1,-0.42,0.59,Boston Celtics,2012


In [ ]:
corr

In [ ]:
normalized.plot(x = 'b2b_away_pct', y = 'b2b_away_games', kind = 'scatter')